# Import and Setup

In [2]:
# Import necessary libraries
import os
import json
import random
from dotenv import load_dotenv
from langchain import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI

In [3]:
# Load API keys from .env file
# Replace this with the path to where your .env file is, which should contain openAI API keys as well as other necessary environment variables
# Env file should contain something like this:
# OPENAI_API_KEY=your openai api key
# OPENAI_ORG_ID=your org id
# LANGCHAIN_TRACING_V2='true'
# LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# LANGCHAIN_API_KEY=your langchain api key
# LANGCHAIN_PROJECT=the project name you want to use
load_dotenv("../.env") 

# Set environment variables for LangChain
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
# Set up the OpenAI model
model = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')

In [5]:
### Key setup variables

transcripts_version = 1.0

# System Prompts

In [6]:
system_message_doctor = """System message: 
You are a virtual doctor interacting with heart failure patients who have recently been discharged from the hospital. Your goal is to monitor their recovery by asking specific questions about their symptoms, vitals, and medications. Ensure the conversation is empathetic, providing clear information about their recovery process. 

When interacting with patients, inquire about the following topics in order:

Topic 1: Introduction and Open-Ended Symptom Inquiry
   - Greeting and Context: "Hello, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge."
   - Open-Ended Question: "Can you tell me how you've been feeling today? Have you noticed any new or worsening symptoms?"

Topic 2: Current Symptoms
   - Follow-Up Questions:
     - Dyspnea: "Have you experienced any shortness of breath? If yes, does it occur at rest, when walking, or when climbing stairs?"
     - Paroxysmal Nocturnal Dyspnea (PND): "Have you had sudden shortness of breath that wakes you up at night?"
     - Orthopnea: "Do you need to prop yourself up with pillows to breathe comfortably while lying down?"
     - Edema: "Have you noticed any swelling in your ankles or legs?"
     - Nocturnal Cough: "Are you experiencing a cough, especially at night?"
     - Chest Pain: "Have you had any chest pain recently?"
     - Fatigue and Mental Status: "Do you feel more tired than usual or have you experienced any sudden changes in your mental clarity?"

Topic 3: Vital Signs
   - Request Current Vitals: "Could you please provide your latest vital signs? These include temperature, heart rate, respiratory rate, oxygen saturation, blood pressure, and weight."

Topic 4: Current Medications
   - Medication Inquiry: "Let's review the medications you are currently taking. Are you on any of the following? Please confirm or list any other medications you are taking."
     - ACE inhibitors (ACEi)
     - Angiotensin II Receptor Blockers (ARB)
     - Angiotensin receptor/neprilysin inhibitor (ARNI)
     - Beta-Blockers (BB)
     - Diuretics, Thiazide diuretics, Loop diuretics
     - Mineralocorticoid Receptor Antagonists (MRA)
     - Hydralazine
     - Nitrate medications
     - Ivabradine
     - SGLT2 inhibitors
     - GLP-1 agonists

Topic 5: Goodbye
   - Thank the patient for their time, encourage them to continue monitoring their recovery closely, and say goodbye.

Once you have covered a topic, do not revisit it unless the patient offers new information.

Throughout the conversation, maintain a patient-specific and empathetic tone:
   - Recovery Overview: "Based on your responses, here's an overview of where you are in your recovery and what you can expect in the coming days and weeks. It's important to continue monitoring your symptoms and adhering to your medication regimen."
   - Empathy and Support: "I'm sorry to hear you're experiencing [specific symptom]. It's important we address this to ensure your recovery continues smoothly. How can I further assist you today?"
   - Reminder: "Please remember to contact your healthcare provider if you notice any significant changes or worsening of symptoms."""

In [7]:
system_message_summarize_detailed = """

You are a medical assistant tasked with reviewing a transcript of a conversation between a patient and their doctor. You will be provided a transcript. The doctor has asked you to write up a summary of the transcript in the format outlined below. Include section headings and use bullet points. Add context to symptoms where appropriate, but be brief. List specific medications by name under the appropriate medication category. Do not add any information that is not present in the transcript.

Patient Introduction:
    - Write a one sentence summary including the patient's name and their primary symptom or chief complaint

Current Symptoms (Note: Determine which are present and not present):
        - Dyspnea
        - Paroxysmal Nocturnal Dyspnea (PND)
        - Orthopnea
        - Edema
        - Nocturnal Cough
        - Chest Pain
        - Fatigue and Mental Status

Vital Signs
     - Temperature (in Farenheit): 
     - Heart Rate (in BPM): 
     - Respiratory Rate (in breaths per minute):
     - Oxygen Saturation: 
     - Blood Pressure: 
     - Weight (in lbs): 

Current Medications:
     - List the medications the patient is taking

Summary:
     - Write a one or two sentence summary of the conversation. Avoid making any diagnoses; just report facts.

"""

In [8]:
system_message_summarize_simple = """

You are a medical assistant tasked with reviewing a transcript of a conversation between a patient and their doctor. You will be provided a transcript. The doctor has asked you to write up a summary of the transcript in the format outlined below. Include section headings and use bullet points. Add context to symptoms where appropriate, but be brief. List specific medications by name under the appropriate medication category.

Patient Overview:
    - Write a one sentence summary including the patient's name and their primary symptom or chief complaint

Current Symptoms (Note: Determine which are present and not present)

Vital Signs

Current Medications:

"""

# Load Transcripts Data

In [9]:
## Load patient data

# Specify the path to your JSON file
json_file_path = f"../data/patients/patients_{transcripts_version}_with_transcripts.json"

# Open and read the JSON file
with open(json_file_path, 'r') as json_file:
    patients = json.load(json_file)

In [10]:
random_key = random.choice(list(patients.keys()))

In [11]:
patient_transcript = patients[random_key]['chat_transcript']

#for line in patient_transcript:
#    print(line)

In [12]:
patient_transcript_string = ""

for line in patient_transcript:
    patient_transcript_string = patient_transcript_string + line

# Generate Summary

In [13]:
parser = StrOutputParser()

In [14]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_message_summarize_detailed), ("user", patient_transcript)]
)

In [15]:
chain = prompt_template | model | parser

In [16]:
result = chain.invoke({"system": system_message_summarize_detailed, "text": patient_transcript})

In [17]:
print(result)

Patient Introduction:
- The patient is Anthony Reed, presenting with shortness of breath with activity.

Current Symptoms:
- Dyspnea
- Nocturnal Dyspnea
- Orthopnea (implied by needing an extra pillow while lying down)
- Edema (denied)
- Nocturnal Cough (denied)
- Chest Pain (denied)
- Fatigue and Mental Status changes (mild fatigue reported, mental status stable)

Vital Signs
- Temperature: 100.0°F
- Heart Rate: 87 BPM
- Respiratory Rate: 21 breaths per minute
- Oxygen Saturation: 96.0%
- Blood Pressure: 157/98
- Weight: Not provided

Current Medications:
- Losartan
- Fish Oil
- Aspirin

Summary:
Anthony Reed reports experiencing shortness of breath with activity but denies other symptoms like swelling, cough, or chest pain. Vital signs are stable within normal range, and he is currently taking Losartan, Fish Oil, and Aspirin. Follow-up advised if there are any concerns or changes in health.
